3) Most typical documents for topics

a) For the experience conducted in 2) c) add a new step by modifying again the CoherenceTestIwor.py including stopwords removal to highlight which document is the most typical of each topic. For that purpose you can rely on topic proportion (or probability) in documents (looking to gensim page https://radimrehurek.com/gensim/models/ldamodel.html) or on measuring cosine similarity (looking to papers or Web infos on that topic) between topic profile (list of topic words) and document profiles (list of document words).
Present the new program and the obtained results.


In [17]:
import nltk
from gensim.corpora.dictionary import Dictionary
from nltk.corpus import stopwords
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

nltk.download('stopwords')

texts = []
file = open("TokenVieuxM.txt", "r")
lines = file.readlines()
file.close()

for line in lines:
    line = line.strip()
    lt = line.split(",")
    # Potential ill-character cleaning
    for i in range(len(lt)):
        lt[i] = lt[i].replace('[','')
        lt[i] = lt[i].replace(']','')
        lt[i] = lt[i].replace('"','')
        lt[i] = lt[i].replace('\n','')
        lt[i] = lt[i].replace(' ', '')

    ltc = [word for word in lt if not word in stopwords.words()]
    texts.append(ltc)

nb = 10
id2word = Dictionary(texts)
corpus = [id2word.doc2bow(text) for text in texts]

# Train the LDA model on the corpus
from gensim.models import LdaModel

lda = LdaModel(corpus, num_topics=nb)

# Print topic description
for i in range(0, nb):
    value = lda.get_topic_terms(i)
    print("Topic", i)
    for j in value:
        print(id2word[j[0]], "- P=", j[1])
    print()

# Compute Perplexity
perplexity_lda = lda.log_perplexity(corpus)
print('Perplexity:', perplexity_lda)

# Compute Coherence Score
from gensim.models import CoherenceModel

coherence_model_lda = CoherenceModel(model=lda, texts=texts, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('Coherence:', coherence_lda)

# Identify most typical document for each topic
print("\nMost Typical Documents:\n")

# Create topic profiles
topic_profiles = np.zeros((nb, len(id2word)))
for i in range(nb):
    topic_terms = lda.get_topic_terms(i)
    for term in topic_terms:
        term_id = term[0]
        term_prob = term[1]
        topic_profiles[i][term_id] = term_prob

for i in range(nb):
    print("Topic", i)
    topic_profile = topic_profiles[i].reshape(1, -1)
    doc_profiles = np.zeros((len(corpus), len(id2word)))

    # Create document profiles
    for j, doc in enumerate(corpus):
        doc_terms = [term[0] for term in doc]
        doc_probs = [term[1] for term in doc]
        doc_profiles[j][doc_terms] = doc_probs

    similarities = cosine_similarity(topic_profile, doc_profiles).flatten()
    most_typical_doc_id = np.argmax(similarities)
    most_typical_doc = lines[most_typical_doc_id].strip()
    print("Document ID:", most_typical_doc_id)
    print("Document:", most_typical_doc)
    print()


[nltk_data] Downloading package stopwords to /home/dario/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Topic 0
months - P= 0.01300052
ice - P= 0.0122842165
neuronal - P= 0.010051842
appv717f - P= 0.010019071
show - P= 0.009917899
numbers - P= 0.009168252
il1β - P= 0.009078606
plaques - P= 0.008878102
mice - P= 0.008731241
kda - P= 0.008297785

Topic 1
auditory - P= 0.022890842
neurons - P= 0.016073532
individual - P= 0.009477287
– - P= 0.008473006
tegmental - P= 0.0084218215
neural - P= 0.008257722
laminar - P= 0.008091558
nucleus - P= 0.00784895
targets - P= 0.0077765896
biological - P= 0.0077317827

Topic 2
structural - P= 0.026491009
husbands - P= 0.021528274
strategies - P= 0.019732
work - P= 0.017551584
experiences - P= 0.015982289
gendered - P= 0.012127318
deal - P= 0.011444971
approach - P= 0.010860704
gender - P= 0.010158007
tie - P= 0.010034637

Topic 3
dna - P= 0.013774116
neurons - P= 0.008261708
months - P= 0.007677997
analysis - P= 0.007598547
ice - P= 0.007329048
age - P= 0.00696969
evidence - P= 0.006855811
kda - P= 0.0068355906
cell - P= 0.0067923274
alzheimer’s - P= 0.0